In [20]:
# colab : %tensorflow_version 1.x

import tensorflow as tf
print(tf.__version__)

1.15.0


### 데이터 준비

In [21]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [22]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

### CNN 계층 구성
* tf.random_normal([3, 3, 1, 32]) -> 3x3 필터, 1: 채널(하나의 색), 32개의 필터개수
* strides=[1,1,1,1] : 필터의 이동. 양 끝의 1은 고정.
* tf.nn.conv2d : 컨볼루션
* tf.nn.max_pool : 풀링
    * ksize : 풀링의 필터 크기
    * strides : 이동

In [23]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)   # 활성화함수 relu : 음수의 값은 0으로 만들어서 영향력 제거. 양수는 비례하게
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')   # padding : 이미지의 크기를 유지
print(L1)

Tensor("MaxPool_3:0", shape=(?, 14, 14, 32), dtype=float32)


In [24]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)   # 활성화함수 relu : 음수의 값은 0으로 만들어서 영향력 제거. 양수는 비례하게
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')   # padding : 이미지의 크기를 유지
print(L2)

Tensor("MaxPool_4:0", shape=(?, 7, 7, 64), dtype=float32)


* 앞의 결과가 노드수가 7 X 7 X 64 이므로, tf.reshape함수를 이용. 1차원으로 만듬.(FCL 입력)
* 인접한 계층의 모든 뉴런과 상호 연결된 계층을 완전 연결 계층(fully connect layer)라 한다
* FCL의 은닉층의 뉴런수는 256개.

In [25]:
W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
print(L3)
L3 = tf.matmul(L3,W3)
L3 = tf.nn.relu(L3)
print(L3)

Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
Tensor("Relu_6:0", shape=(?, 256), dtype=float32)


### 최종 출력값 L3 에서의 출력 256개를 입력값으로 받아서 0~9 레이블인 10개의 출력값 만들기

In [26]:
W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
model = tf.matmul(L3,W4)
print(model)

Tensor("MatMul_3:0", shape=(?, 10), dtype=float32)


### 비용함수, 최적화 함수 구성
* 비용함수와 최적화 함수 지정
* 비용함수 : softmax_cross_entropy_with_logits_v2
* 최적화 함수 : AdamOptimizer

In [27]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,
labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

### 신경망 모델 학습

In [28]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

### 1 epoch(데이터 전체 학습 한번)만 돌리기

In [30]:
%%time

total_cost = 0
epoch = 0
for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성
    batch_xs = batch_xs.reshape(-1, 28, 28, 1)
    
    _, cost_val = sess.run([optimizer, cost],
                            feed_dict={X: batch_xs, Y: batch_ys})
    total_cost += cost_val
    
print('Epoch:', '%04d' % (epoch + 1),
        'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!(1 epoch)')

Epoch: 0001 Avg. cost = 0.325
최적화 완료!(1 epoch)
Wall time: 40.3 s
